In [1]:
import numpy
import math 
from matplotlib import pyplot
%matplotlib inline

In [2]:
N = 100 
x_start, x_end = -1.0, 1.0
y_start, y_end = -1.5, 1.5
x = numpy.linspace(x_start, x_end, N)
y = numpy.linspace(y_start, y_end, N)
X,Y = numpy.meshgrid(x,y)

In [3]:
u_inf = 1.0

u_freestream = u_inf * numpy.ones((N,N), dtype=float)
v_freestream = numpy.zeros((N,N), dtype=float)

In [4]:
class Source: 
    
    def __init__(self, strength, x, y):
        
        self.strength = strength
        self.x, self.y = x,y
        
    def velocity(self, X, Y):
        
        self.u = self.strength/(2*math.pi)*(X-self.x)/((X-self.x)**2 + (Y-self.y)**2)
        self.v = self.strength/(2*math.pi)*(Y-self.y)/((X-self.x)**2 + (Y-self.y)**2)
        
    def stream_function(self, X, Y):
        
        self.psi = self.strength/(2*math.pi)*numpy.arctan2((Y-self.y), (X-self.x))
        
    

In [6]:
N_sources = 11
strength = 5.0 
strength_source = strength/N_sources
x_source = numpy.zeros(N_sources, dtype=float)
y_source = numpy.linspace(-1.0, 1.0, N_sources)

sources = numpy.empty(N_sources, dtype=object)
for i in range(N_sources):
    sources[i] = Source(strength_source, x_source[i], y_source[i])
    sources[i].velocity(X, Y)
    
u = u_freestream.copy()
v = v_freestream.copy()
for source in sources:
    u += source.u
    v += source.v